In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import time
from datetime import datetime
import datetime as dt
import warnings
warnings.simplefilter("ignore")

In [2]:
# output file: HK_ID_YYMMDD.xlsx
current_date = time.strftime("%Y%m%d")
report_name = 'HK_ID_' + current_date
#change path below to folder you would like to save the output file
drive_letter = r'..\Python Scripts\\'
folder_to_save_files = drive_letter + report_name + '.xlsx'

In [3]:
# import the party_data.xlsx
file_path = r'108536_Report_2.xlsx'
sheet_name = '108536 Report'
df = pd.read_excel(file_path, sheet_name = sheet_name, na_filter = False)
# filter for individuals
df_natural = df[df['Party Type']=='Natural Person']

In [4]:
#Preprocessing of party name
df_natural.loc[:,'Party Name_pre']=df_natural['Party Name'].copy()
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.strip()
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.replace(".","")
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.replace(" ","")
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.replace('\t','')
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.replace(",","")
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str.lower()

#Prepocessing 'Identification Document Number' to remove space & casing
df_natural.loc[:,'Identification Document Number_pre']=df_natural['Identification Document Number'].copy()
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str.replace(".","")
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str.replace(" ","")
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str.replace(",","")
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str.lower()

# beneficiary only list
bene_only = df_natural.groupby('Party ID')['Relationship (Party Role)'].agg(set).reset_index(name='role_set')
bene_only = bene_only[bene_only['role_set']=={'Beneficiary'}]
bene_only_list = bene_only['Party ID'].to_list()
df_natural['bene_only'] = df_natural['Party ID'].isin(bene_only_list)

#dates
df_natural['Date of Birth']  = pd.to_datetime(df_natural['Date of Birth'], format='%Y-%m-%d', errors='coerce').dt.date

### < Specific Code Goes Here >
HK Nationality 
No HK Passport
HK ID or HK Permanent ID

In [30]:
tab25 = df_natural.copy()

# create a key
# should the key be party name-DOB or party ID
tab25['key'] = tab25['Party Name_pre'].astype(str) + tab25['Date of Birth'].astype(str)

# only HK Nationality
tab25 = tab25[tab25["Nationality"]=="Hong Kong"]

# only HK documents
tab25 = tab25[tab25["Identification Document Issue Country"]=="Hong Kong"]

# set of all ID documents a party has
tab25_id = tab25.groupby('key')['Identification Document Type'].agg(set).reset_index(name='id_set')
id_set = tab25_id['id_set'].tolist()

# function to check that id set does not contain HK passport
def check_id(id_set):
    hk_set = {'Passport'}
    result = id_set.intersection(hk_set)
    return len(result) == 0 #if TRUE: empty intersection. means HK Passport not present

tab25_id['passport_missing'] = tab25_id['id_set'].apply(lambda x: check_id(x))
tab25_id = tab25_id[tab25_id['passport_missing']==True]

# compare keep list to original table
tab25_id_list = tab25_id['key'].tolist()
tab25['exist'] = tab25['key'].isin(tab25_id_list)
tab25 = tab25[tab25['exist']==True]

# remove duplicates and sort
tab25 = tab25.drop_duplicates(subset=['Party ID', 'key'])
tab25 = tab25.sort_values(by=["key",'Party ID'])

# select columns
tab25_columns = tab25[['Party ID','Party Name','RM UID.1','RM Name.1','Date of Birth','Nationality','Identification Document Type','Identification Document Issue Country','bene_only']]

In [31]:
tab25_columns.to_csv('HK_ID.csv', index=False)